In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'InSlateAttentionSequencewiseGRU'
device = 'cuda:2'
seed = 123
pkl_path = '../data/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.9.0'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class AttentionResponseModel(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention = torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(3 * embedding.embedding_dim, output_dim)
    
    
    def get_attention_embeddings(self, item_embs, user_embs, slate_mask):
        shp = item_embs.shape      
        key_padding_mask = slate_mask
        key_padding_mask[:,:, 0] = True # let model attent to first padd token if slate is empty 
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, item_embs.size(-2), 1).reshape(shp)
            ],
            dim = -1
        ).flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        shp = list(shp)
        shp[-1] *= 2
        features = features.reshape(shp)
        return features
    
    def forward(self, batch):
        # consider sequential clicks, hence need to flatten slates
        item_embs, user_embs = self.embedding(batch)
        slate_mask = batch['slates_mask'].clone()
        
        att_features = self.get_attention_embeddings(item_embs, user_embs, slate_mask)
        
        gru_features, _ = self.rnn_layer(item_embs.flatten(-3, -2))
        gru_features = gru_features.reshape(item_embs.shape)
        
        features = torch.cat(
            [att_features, gru_features],
            dim=-1
        )
        
        return self.out_layer(features).squeeze(-1)


    
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)
emb = RecsysEmbedding(d.n_items, dummy_matrix, embeddings='explicit')

for batch in dummy_loader:
    break

model = AttentionResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural'),
    output_dim=1
).to('cpu')

model(batch)


3it [00:00, 3044.50it/s]

biulding affinity matrix...


tensor([[[ 0.1783,  0.1444,  0.1578],
         [ 0.1012,  0.0908,  0.1225]],

        [[-0.2847, -0.2641, -0.2120],
         [-0.0057,  0.0075,  0.0132]]], grad_fn=<SqueezeBackward1>)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

for embeddings in ['explicit', 'svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionResponseModel(
        RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
        output_dim=1
    ).to('cpu')
    _, metrics = train(model, 
       dummy_loader, dummy_loader, dummy_loader, device=device, lr=1e-3, num_epochs=5000, dummy=True,
       silent=True,
#        debug=True,
    
    )


3it [00:00, 3583.85it/s]

biulding affinity matrix...

Evaluating InSlateAttentionSequencewiseGRU with explicit embeddings



train... loss:0.7021339535713196:   0%|                                                                                      | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.6666666269302368, 'accuracy': 0.25}


train... loss:0.670462965965271:   0%|                                                                               | 1/5000 [00:00<31:28,  2.65it/s]

...

Test before learning: {'f1': 0.0, 'roc-auc': 1.0, 'accuracy': 0.75}


train... loss:0.647767186164856:   0%|                                                                               | 1/5000 [00:00<30:33,  2.73it/s]

Test update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.46


train... loss:0.5346876382827759:   0%|                                                                              | 6/5000 [00:02<29:21,  2.84it/s]

Test update: epoch: 6 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.56


# ContentWise

In [5]:
# c = ContentWise('../data/CW/data/ContentWiseImpressions/CW10M-CSV/')

In [6]:
# c.dump(os.path.join(pkl_path, 'cw.pkl'))

In [7]:
content_wise_results = []
c = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
c_train_loader, c_val_loader, c_test_loader, c_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(c, batch_size=50)
len(c_train_loader), len(c)

(324, 20216)

In [8]:
for embeddings in ['neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, c_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       c_train_loader, c_val_loader, c_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)
    
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')


Evaluating InSlateAttentionSequencewiseGRU with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.16915073990821838, 'roc-auc': 0.5131357908248901, 'accuracy': 0.5203292369842529}


train... loss:105.64544066786766:   0%|                                                                         | 1/5000 [01:43<144:07:24, 103.79s/it]

Test update: epoch: 0 |accuracy: 0.7411112785339355 | f1: 0.2638704478740692 | auc: 0.6752206087112427 | treshold: 0.12


...

train... loss:85.89792414009571:   0%|                                                                           | 8/5000 [11:44<121:47:58, 87.84s/it]

Test update: epoch: 7 |accuracy: 0.8774176239967346 | f1: 0.3114840090274811 | auc: 0.7526431679725647 | treshold: 0.18000000000000002


train... loss:85.33268457651138:   0%|▏                                                                         | 11/5000 [16:08<122:06:58, 88.12s/it]

Test update: epoch: 10 |accuracy: 0.8279832601547241 | f1: 0.33439701795578003 | auc: 0.7530032396316528 | treshold: 0.19


train... loss:79.79979886114597:   1%|▌                                                                         | 36/5000 [54:02<124:11:55, 90.07s/it]


# RL4RS

In [9]:
# r = RL4RS('../data/rl4rs-dataset/', which='rl4rs_dataset_b_rl.csv', min_session_len=3)
# r.dump(os.path.join(pkl_path, 'rl4rs.pkl'))

In [10]:
rl4rs_results = []
r = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
r_train_loader, r_val_loader, r_test_loader, r_train_user_item_matrix, train_num_items = get_train_val_test_tmatrix_tnumitems(r, batch_size=20000)
len(r_train_loader), len(r)

(2, 45942)

In [11]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, r_train_user_item_matrix, embeddings='neural'),
        output_dim=1
    ).to(device)

    _, metrics = train(model, 
       r_train_loader, r_val_loader, r_test_loader, device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)
    
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')


Evaluating InSlateAttentionSequencewiseGRU with neural embeddings


train:   0%|                                                                                                                 | 0/5000 [00:00<?, ?it/s]

Test before learning: {'f1': 0.4037405550479889, 'roc-auc': 0.509438693523407, 'accuracy': 0.4434046745300293}


train... loss:1.4017160534858704:   0%|                                                                           | 1/5000 [00:56<77:57:09, 56.14s/it]

Test update: epoch: 0 |accuracy: 0.6443960666656494 | f1: 0.7837225794792175 | auc: 0.5602662563323975 | treshold: 0.38


...

Test update: epoch: 21 |accuracy: 0.715076744556427 | f1: 0.8103218078613281 | auc: 0.7702997326850891 | treshold: 0.22


train... loss:1.1847588419914246:   0%|▎                                                                          | 23/5000 [04:11<8:37:54,  6.24s/it]

Test update: epoch: 22 |accuracy: 0.7149316668510437 | f1: 0.8109737634658813 | auc: 0.7761297225952148 | treshold: 0.23


train... loss:1.159471035003662:   0%|▎                                                                          | 23/5000 [04:17<15:29:19, 11.20s/it]

Test update: epoch: 23 |accuracy: 0.7169628739356995 | f1: 0.8120675086975098 | auc: 0.7820822596549988 | treshold: 0.26
